## 1. Settings

### 1) Import required libraries

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

### 2) Set hyperparameters

In [2]:
batch_size = 16
learning_rate = 0.0002
num_epoch = 10

## 2. Data

### 1) Download Data

In [3]:
mnist_train = dset.MNIST("./", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = dset.MNIST("./", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)
#MNIST 데이터를 다운로드 받는다.

### 2) Check Dataset

In [4]:
print(mnist_train.__getitem__(0)[0].size(), mnist_train.__len__())
mnist_test.__getitem__(0)[0].size(), mnist_test.__len__()

#mnist_train도 Dataset type이다. 따라서, len, getitem 메서드를 사용할 수 있다.

torch.Size([1, 28, 28]) 60000


(torch.Size([1, 28, 28]), 10000)

### 3) Set DataLoader

In [5]:
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

## 3. Model & Optimizer

### 1) CNN Model

In [6]:
from collections import OrderedDict #이건 Python collections 임

# class Flatten(nn.Module): #Sequential을 사용할 때 Flatten
#     def forward(self, x):
#         return x.view(x.size()[0], -1) #class를 따로 만들어 view를 해주면 된다.

# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
        
#         self.model = nn.Sequential(OrderedDict([
#             ("conv1", nn.Conv2d(1, 16, 5)), #Conv2d 모델 (in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
#             ("relu1", nn.ReLU()),
#             ("conv2", nn.Conv2d(16, 32, 5)),
#             ("relu2", nn.ReLU()),
#             ("pool1", nn.MaxPool2d(2, 2)), 
#             ("conv3", nn.Conv2d(32, 64, 5)),
#             ("relu3", nn.ReLU()),
#             ("pool2", nn.MaxPool2d(2, 2)), 
#             ("flatten", Flatten()),
#             ("fc1", nn.Linear(64*3*3, 100)),
#             ("relu4", nn.ReLU()),
#             ("fc2", nn.Linear(100, 10)),
#         ]))
        
#     def forward(self, x):
#         return self.model(x)
    
    
#Conv2d 모델 (in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
#pooling을 지나면, 채널은 그대로이고 사이즈가 준다. 필터의 수 만큼 반복되므로, 필터의 수가 out_channels 이 된다.
    
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,5), # input : 28 * 28 1채널 #output : 24 * 24 16채널 (5 x 5, stride 1이므로 4씩 줄어든다)
            #Conv2d : (in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
            nn.ReLU(),
            nn.Conv2d(16,32,5), # input : 24 * 24 16채널 #output : 20 * 20 32채널 (5 x 5, stride 1이므로 4씩 줄어든다)
            nn.ReLU(),
            nn.MaxPool2d(2,2), # input : 20 * 20 32채널 #output : 10 * 10 32채널 (pooling에선 채널이 그대로 유지된다)
            #MaxPool2d : (kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
            #여기서는 2 * 2 크기로 2칸 씩 이동하므로 절반으로 줄어든다.
            nn.Conv2d(32,64,5), # input : 10 * 10 32채널 #output : 6 * 6 64채널 (5 x 5, stride 1이므로 4씩 줄어든다)
            nn.ReLU(),
            nn.MaxPool2d(2,2) # input : 6 * 6 64채널 #output : 3 * 3 64채널 (pooling에선 채널이 그대로 유지된다)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3,100), #마지막으로 pooling 이후 output이 3 * 3 64채널 이었으므로 이를 FC에 넣어준다.
            nn.ReLU(),
            nn.Linear(100,10)
        )       
        
    def forward(self,x):
        out = self.layer(x)
        out = out.view(batch_size,-1)
        out = self.fc_layer(out)

        return out

model = CNN()

### 2) Loss func & Optimizer

In [7]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## 4. Train

In [ ]:
for i in range(num_epoch):
    for j,[image,label] in enumerate(train_loader):
        optimizer.zero_grad()
        output = model.forward(image)
        loss = loss_func(output, label)
        loss.backward()
        optimizer.step()
        
        if j % 1000 == 0:
            print(loss)

tensor(2.3081, grad_fn=<NllLossBackward>)


## 5. Test

In [ ]:
correct = 0
total = 0

for image,label in test_loader:
    output = model.forward(image)
    _,output_index = torch.max(label,1)
        
    total += label.size(0)
    correct += (output_index == label).sum().float()
    
print("Accuracy of Test Data: {}".format(100*correct/total))

##이미지 데이터에 대해서는 CNN이 Linear보다 확연히 좋은 성능을 보여준다.